[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# BOTSORT IMPLEMENTATION GUIDE

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
[![Scientific Paper](https://img.shields.io/badge/Official-Paper-blue.svg)](<PAPER LINK>)

### About Notebook
This notebook provide you with two method through which you can implement BotSORT on you own.
1. Using Ultralytics :- Default track algo used in yolo is botSORT so using Ultralytics it can easily be implemented

2. Using PyPI - boxmot :- A python library you can use to implement various tracking algo can be use to implement BotSORT

In [1]:
!python -V

Python 3.10.18


In [2]:
!pip install boxmot==13.0.9
!pip install ultralytics==8.3.159

# or 

# !pip install -r requirements.txt


In [ ]:
# !pip freeze > requirements.txt

## Helper function

In [5]:
from IPython.display import Video, display

import os
import requests
from IPython.display import Video, display

def show_mp4(
    url: str,
    local_folder: str = "../videos",
    filename: str    = None,
    width: int       = 640,
    autoplay: bool   = True,
    loop: bool       = True,
    muted: bool      = True,
    timeout: float   = 10.0,
):
    """
    Downloads an MP4 from a CDN URL into a local folder, then embeds it in Colab.

    Args:
        url (str): Full URL to the .mp4 file.
        local_folder (str): Path to folder where you want to save the download.
        filename (str, optional): Name to save the file as; if None, inferred from URL.
        width (int): Display width in pixels.
        autoplay (bool): Start playing automatically.
        loop (bool): Loop playback.
        muted (bool): Mute audio.
        timeout (float): Max seconds to wait for HTTP response.

    Raises:
        ValueError: if the URL doesn’t look like an MP4 or response isn’t correct.
        requests.HTTPError: if download fails (non-2xx status).
    """
    # Infer filename
    if filename is None:
        filename = os.path.basename(url.split("?", 1)[0])
    if not filename.lower().endswith(".mp4"):
        raise ValueError(f"Expected an .mp4 file, got '{filename}'")

    # Ensure folder
    os.makedirs(local_folder, exist_ok=True)
    local_path = os.path.join(local_folder, filename)

    # Download if not already present
    if not os.path.isfile(local_path):
        resp = requests.get(url, stream=True, timeout=timeout)
        resp.raise_for_status()
        content_type = resp.headers.get("Content-Type", "")
        if "video/mp4" not in content_type.lower():
            raise ValueError(f"URL did not return MP4 (Content-Type={content_type})")

        # Stream write to disk
        with open(local_path, "wb") as f:
            for chunk in resp.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
    else:
        print(f"✓ Using cached file at {local_path}")

    # Build video tag attributes
    attrs = []
    if autoplay: attrs.append("autoplay")
    if loop:     attrs.append("loop")
    if muted:    attrs.append("muted")
    html_attrs = " ".join(attrs)

    # Display
    display(Video(local_path, embed=True, width=width, html_attributes=html_attrs))



# Using Ultralytics

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/mall.mp4", width=640)

In [15]:
from ultralytics import YOLO

model = YOLO('yolo12x.pt')  # or your trained model
results = model.track(source='../videos/mall.mp4', persist=True, save=True, classes=[0])


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/341) /home/HP/Notebook/Botsort/../videos/mall.mp4: 384x640 16 persons, 33.5ms
video 1/1 (frame 2/341) /home/HP/Notebook/Botsort/../videos/mall.mp4: 384x640 16 persons, 29.3ms
video 1/1 (frame 3/341) /home/HP/Notebook/Botsort/../videos/mall.mp4: 384x640 17 persons, 28.7ms
video 1/1 (frame 4/341) /home/HP/Notebook/Botsort/../videos/mall.mp4: 384x640 17 persons, 28.8ms
video 1/1 (frame 5/341) /home/HP/Notebook/Botsort/../videos/mall.mp4

# Using PyPI - boxmort

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/crowd.mp4", filename="crowd.mp4",width=800)

In [7]:
# !boxmot track --source ../videos/crowd.mp4 --tracking-method botsort --yolo-model yolo12x.pt --reid-model osnet_x1_0_msmt17.pt --project boxmot_results --name crowd --save --classes 0

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

2025-06-27 07:14:38.521 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v13.0.8 🚀 Python-3.10.18 torch-2.7.1+cu126
CUDA:0 (NVIDIA L4, 22574MiB)
2025-06-27 07:14:38.523 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at osnet_x1_0_msmt17.pt; skipping download.
2025-06-27 07:14:38.662 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x1_0_msmt17.pt
video 1/1 (frame 1/176) /home/HP/Notebook/Botsort/../videos/crowd.mp4: 384x640 10 persons, 106.0ms
video 1/1 (frame 2/176) /home/HP/Notebook/Botsort/../videos/crowd.mp4: 384x640 10 persons, 26.7ms
video 1/1 (frame 3/176) /home/HP/Notebook/Botsort/../videos/crowd.mp4: 384x640 11 persons, 30.0ms
video 1/1 (frame 4/176) /home/HP/Notebook/Botsort/../videos/crowd.mp4: 384x640 11 persons, 28.5ms
vi

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/car.mp4", width=640)

In [10]:
!boxmot track --source ../videos/car.mp4 --tracking-method botsort --yolo-model yolo12x.pt --reid-model osnet_x1_0_msmt17.pt --project boxmot_results --name car --save --gsi --conf 0.1 --iou 0.3 --classes 2

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

2025-06-27 07:16:37.675 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v13.0.8 🚀 Python-3.10.18 torch-2.7.1+cu126
CUDA:0 (NVIDIA L4, 22574MiB)
2025-06-27 07:16:37.676 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at osnet_x1_0_msmt17.pt; skipping download.
2025-06-27 07:16:37.786 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x1_0_msmt17.pt
video 1/1 (frame 1/246) /home/HP/Notebook/Botsort/../videos/car.mp4: 384x640 (no detections), 96.4ms
video 1/1 (frame 2/246) /home/HP/Notebook/Botsort/../videos/car.mp4: 384x640 (no detections), 29.7ms
video 1/1 (frame 3/246) /home/HP/Notebook/Botsort/../videos/car.mp4: 384x640 (no detections), 27.7ms
video 1/1 (frame 4/246) /home/HP/Notebook/Botsort/../videos/car.mp4: 384x640 (no detections)

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/runner.mp4", width=800)

In [11]:
!boxmot track --source ../videos/runner.mp4 --tracking-method botsort --yolo-model yolo12x.pt --reid-model osnet_x1_0_msmt17.pt --project boxmot_results --name runner --save --gsi --conf 0.1 --iou 0.3 --classes 0

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

2025-06-27 07:17:22.152 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v13.0.8 🚀 Python-3.10.18 torch-2.7.1+cu126
CUDA:0 (NVIDIA L4, 22574MiB)
2025-06-27 07:17:22.153 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at osnet_x1_0_msmt17.pt; skipping download.
2025-06-27 07:17:22.266 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x1_0_msmt17.pt
video 1/1 (frame 1/239) /home/HP/Notebook/Botsort/../videos/runner.mp4: 384x640 5 persons, 95.7ms
video 1/1 (frame 2/239) /home/HP/Notebook/Botsort/../videos/runner.mp4: 384x640 5 persons, 27.0ms
video 1/1 (frame 3/239) /home/HP/Notebook/Botsort/../videos/runner.mp4: 384x640 5 persons, 34.7ms
video 1/1 (frame 4/239) /home/HP/Notebook/Botsort/../videos/runner.mp4: 384x640 5 persons, 27.3ms
vid

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/plane.mp4", width=800)

In [12]:
!boxmot track --source ../videos/plane.mp4 --tracking-method botsort --yolo-model yolo12x.pt --reid-model osnet_x1_0_msmt17.pt --project boxmot_results --name plane --save --gsi --conf 0.1 --iou 0.3 --classes 4

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

2025-06-27 07:17:55.063 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v13.0.8 🚀 Python-3.10.18 torch-2.7.1+cu126
CUDA:0 (NVIDIA L4, 22574MiB)
2025-06-27 07:17:55.063 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at osnet_x1_0_msmt17.pt; skipping download.
2025-06-27 07:17:55.178 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x1_0_msmt17.pt
video 1/1 (frame 1/208) /home/HP/Notebook/Botsort/../videos/plane.mp4: 384x640 5 airplanes, 96.2ms
video 1/1 (frame 2/208) /home/HP/Notebook/Botsort/../videos/plane.mp4: 384x640 5 airplanes, 27.5ms
video 1/1 (frame 3/208) /home/HP/Notebook/Botsort/../videos/plane.mp4: 384x640 5 airplanes, 27.1ms
video 1/1 (frame 4/208) /home/HP/Notebook/Botsort/../videos/plane.mp4: 384x640 5 airplanes, 27.0ms

In [ ]:
show_mp4("https://storage.googleapis.com/labellerr-cdn/%200%20Sample-videos/birds.mp4", width=640)

In [ ]:
!boxmot track --source ../videos/birds.mp4 --tracking-method botsort --yolo-model yolo12x.pt --reid-model osnet_x1_0_msmt17.pt --project boxmot_results --name birds@rf --save --gsi --conf 0.1 --iou 0.3 --classes 14